In [1]:
!pip install numpy
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Lista de frameworks e bibliotecas

$$ \frac {x_1}{x_2} $$

In [192]:
import numpy as np
import pandas as pd
import yfinance as yf # https://github.com/ranaroussi/yfinance
from multipledispatch import dispatch
import logging
import os

# Environment Variable

In [193]:
STOCK_NAME = "USDBRL=X"
ASSETS_URL = "assets/"
FILE_NAME = "usd2brl.csv"
FILE_URL = ASSETS_URL + FILE_NAME
LOG_PATH = "logs/"

# Logs

In [200]:
def logging_basic_config(path):
    if not os.path.exists(path):
        os.mkdir(path)
        
    logging.basicConfig(
        handlers = [
            logging.FileHandler(f'{path}{datetime.now()}.log'),
            logging.StreamHandler()
        ],
        format="%(asctime)s - %(levelname)s - %(message)s",
        level=logging.DEBUG,
        datefmt="%d-%b-%y %H:%M:%S",
    )

In [201]:
logging_basic_config(path=LOG_PATH)

# Obtendo dados

In [202]:
class Database(yf.Ticker):
    def __init__(self, stock_name: str):
        self.stock_name = stock_name
        
    def connect(self):
        try:
            self.database = yf.Ticker(self.stock_name)
        except OverflowError as e:
            raise e
    
    def set_period(self, period: str):
        self.period = period
        
    def set_interval(self, interval: str):
        self.interval = interval
        
    @dispatch()
    def get_data(self) -> pd.DataFrame:
        try:
            dataframe =  pd.DataFrame(self.database.history(period=self.period, interval=self.interval))
            dataframe = dataframe.reset_index()
            try:
                return dataframe[['Date', 'Open','High','Low','Close']]
            except:
                return dataframe[['Datetime',  'Open','High','Low','Close']]
        except:
            pass
    
    @dispatch(datetime, datetime)
    def get_data(self, start_date: datetime, end_date: datetime):
        try: 
            dataframe = self.database.history(period=self.period, interval=self.interval, start=start_date, end=end_date)
            dataframe = dataframe.reset_index()
            try:
                return dataframe[['Date', 'Open','High','Low','Close']]
            except:
                return dataframe[['Datetime',  'Open','High','Low','Close']]
        except :
            pass
    
    def export_csv(self, path: str, file_name: str):
        
        pathfile = path + file_name
        
        dataframe = self.get_data()
        if not os.path.exists(path):
            os.mkdir(path)
        
        files_present = os.path.isfile(pathfile) 
        if not files_present:
            loggin.info(f"The file {file_name} is being written")
            dataframe.to_csv(pathfile)
        else:
            logging.info("The file already exists, it will be deleted and rewritten")
            os.remove(pathfile)
            dataframe.to_csv(pathfile)

In [203]:
database = Database(stock_name=STOCK_NAME)
database.connect()
database.set_period('7d')
database.set_interval('1m')

In [204]:
print(FILE_URL)
database.export_csv(ASSETS_URL, FILE_NAME)

assets/usd2brl.csv


# Carregando dados CSV